In [ ]:
class MajorityChecker:

    def __init__(self, arr: List[int]):
        self.n = len(arr)
        self.arr = arr
        self.vals = [0] * (4 * self.n)  # maj, score
        self._build(1, 0, self.n - 1)
        self.ha = defaultdict(list)
        for i, x in enumerate(arr):
            self.ha[x].append(i)

    def _build(self, o, l, r):  # 初始化
        if l == r:  # basecase
            self.vals[o] = (self.arr[l], 1)
            return self.vals[o]
        # 左右子问题 摩尔投票 得到主问题
        m = l + r >> 1
        x, xc = self._build(o * 2, l, m)
        y, yc = self._build(o * 2 + 1, m + 1, r)
        if x == y: num, cnt = x, xc + yc
        elif xc > yc: num, cnt = x, xc - yc
        else: num, cnt = y, yc - xc
        self.vals[o] = (num, cnt)
        return self.vals[o]

    def _query(self, o, l, r, L, R):  # 查询[L,R]的val
        if r < L or l > R: return (0, 0)  # 没有交集 无关紧要值
        if L <= l and r <= R: return self.vals[o]  # 内部
        # 有交集
        m = l + r >> 1
        x, xc = self._query(o * 2, l, m, L, R)
        y, yc = self._query(o * 2 + 1, m + 1, r, L, R)
        if x == y: num, cnt = x, xc + yc
        elif xc > yc: num, cnt = x, xc - yc
        else: num, cnt = y, yc - xc
        return (num, cnt)

    def query(self, left: int, right: int, threshold: int) -> int:
        num, _ = self._query(1, 0, self.n - 1, left, right)
        pos = self.ha[num]
        d = bisect_right(pos, right) - bisect_left(pos, left)
        if d >= threshold: return num
        return -1


# Your MajorityChecker object will be instantiated and called as such:
# obj = MajorityChecker(arr)
# param_1 = obj.query(left,right,threshold)

In [ ]:
class Node:

    def __init__(self, si, ei, num=0, cnt=0):
        self.si = si
        self.ei = ei
        self.left = self.right = None
        self.val = (num, cnt)  # num cnt 擂台元素 及数量


class SegTree:

    def __init__(self, data) -> None:
        self.data = data
        self.root = self._build(0, len(self.data) - 1)

    def _build(self, l, r) -> Node:
        if l == r:
            return Node(l, r, self.data[l], 1)
        # 创建当前节点
        o = Node(l, r)
        m = l + r >> 1
        # 创建左右节点
        o.left = self._build(l, m)
        o.right = self._build(m + 1, r)
        # 更新节点信息(两侧都用摩尔投票法进行多数元素查询)
        x, xc = o.left.val
        y, yc = o.right.val
        if x == y: num, cnt = x, xc + yc
        elif xc > yc: num, cnt = x, xc - yc
        else: num, cnt = y, yc - xc
        o.val = (num, cnt)
        return o

    def query(self, node, l, r):  # 查询区间
        if r < node.si or l > node.ei:
            return (node.val[0], 0)  # 无影响的值
        if l <= node.si and node.ei <= r:
            return node.val
        x, xc = self.query(node.left, l, r)
        y, yc = self.query(node.right, l, r)
        if x == y: return (x, xc + yc)
        elif xc > yc: return (x, xc - yc)
        else: return (y, yc - xc)


class MajorityChecker:

    def __init__(self, arr: List[int]):
        self.st = SegTree(arr)
        self.ha = defaultdict(list)
        for i, x in enumerate(arr):
            self.ha[x].append(i)

    def query(self, left: int, right: int, threshold: int) -> int:
        x, _ = self.st.query(self.st.root, left, right)
        pos = self.ha[x]
        d = bisect_right(pos, right) - bisect_left(pos, left)
        if d >= threshold: return x
        return -1


# Your MajorityChecker object will be instantiated and called as such:
# obj = MajorityChecker(arr)
# param_1 = obj.query(left,right,threshold)